# Week5 Assignments (part2)
This is the second part of this week's assignments. In this part, you will load the KFP components that you created in the first part of the assignments and use them to create a KFP pipeline.

In [1]:
import kfp
import kfp.dsl as dsl 
import kfp.components as components
import kfp.kubernetes as kubernetes
import os
from utils.send_requests import send_requests

# Connect to KFP client, remember to ensure you're using the correct kubectl context
kfp_client = kfp.Client(host=None)

/home/kaisaeko/anaconda3/envs/mlops_eng2/lib/python3.11/site-packages/kfp/client/client.py:158: FutureWarning: This client only works with Kubeflow Pipeline v2.0.0-beta.2 and later versions.
  warnings.warn(


Let's first load the KFP components from their YAML files you created in the first part of the assignments.

In [2]:
pull_data = components.load_component_from_file(os.path.join("components", "pull_data.yaml"))
preprocess_data = components.load_component_from_file(os.path.join("components", "preprocess_data.yaml"))
hpo = components.load_component_from_file(os.path.join("components", "hpo.yaml"))
train = components.load_component_from_file(os.path.join("components", "train.yaml"))
deploy_model = components.load_component_from_file(os.path.join("components", "deploy_model.yaml"))


## Assignment 2:  Create a KFP pipeline (2 points)
You need to create a KFP pipeline using all the KFP components you created. 

The KFP components should perform their tasks in the following order: 

<img src="./images/kfp-assignment.png" />

(Other inputs needed by the components are provided as arguments of the `pipeline` function.)

The **train** and **deloy_model** components should only perform their tasks if the best MAE found by the **hpo** component is strictly lower than a given threshold. In other words, the pipeline will stop after the HPO task if the best MAE is larger than the given threshold. You can use [dsl.Condition](https://kubeflow-pipelines.readthedocs.io/en/sdk-2.0.1/source/dsl.html#kfp.dsl.Condition) to manage conditions in Kubeflow Pipelines. 

**Hints**: 
- You need to assign the needed credentials to the model training task so that the task can upload artifacts to MLflow's artifact store (MinIO). (Please check the tutorial for more details.) 
- If a component has multiple Parameter outputs, then a single output can be accessed by `task.outputs[output_name]`.

In [3]:
@dsl.pipeline(
    name="bike-demand-pipeline",
    description="An example pipeline that deploys a model for bike demanding prediction",
)
def pipeline(
    url: str,
    random_seed: int,
    hpo_trials: int,
    mae_threshold: float,
    mlflow_experiment_name: str,
    mlflow_tracking_uri: str,
    mlflow_s3_endpoint_url: str,
    model_name: str,
):
    """
    Args:
        url: URL for downloading the dataset
        random_seed: Random seed used for model training and the TPESampler in HPO
        hpo_trials: The number of trials that the HPO component should perform
        mae_threshold: The threshold for the MAE metric. If the best MAE found by the HPO is greater than this threshold, the train and deploy_model components will be skipped
        mlflow_experiment_name: Name of the MLflow experiment
        mlflow_tracking_uri: URI of MLflow's tracking server
        mlflow_s3_endpoint_url: URL of MLflow's artifact store
        model_name: The name of the KServe inference service. It's also used as the model's artifact path
    """
    ### START CODE HERE
    pull_task = pull_data(url=url)
    preprocess_task = preprocess_data(data=pull_task.outputs["data"])

   
    hyperparams = hpo(train_x_csv=preprocess_task.outputs["train_x_csv"],
        train_y_csv=preprocess_task.outputs["train_y_csv"],
        test_x_csv=preprocess_task.outputs["test_x_csv"],
        test_y_csv=preprocess_task.outputs["test_y_csv"],
        hpo_trials=hpo_trials,
        random_seed=random_seed,
        )

    with dsl.Condition(hyperparams.outputs["best_mae"] <= mae_threshold):
        train_task = train(
            train_x_csv=preprocess_task.outputs["train_x_csv"],
            train_y_csv=preprocess_task.outputs["train_y_csv"],
            test_x_csv=preprocess_task.outputs["test_x_csv"],
            test_y_csv=preprocess_task.outputs["test_y_csv"],
            mlflow_experiment_name=mlflow_experiment_name,
            mlflow_tracking_uri=mlflow_tracking_uri,
            mlflow_s3_endpoint_url=mlflow_s3_endpoint_url,
            model_artifact_path=model_name,
            hyperparams=hyperparams.outputs["hyperparams"],
        )

        train_task = kubernetes.use_secret_as_env(
            train_task,
            secret_name="aws-secret",
            secret_key_to_env={
                "AWS_ACCESS_KEY_ID": "AWS_ACCESS_KEY_ID",
                "AWS_SECRET_ACCESS_KEY": "AWS_SECRET_ACCESS_KEY",
            },
        )

        deploy_model(model_name=model_name,storage_uri=train_task.output)
    ### END CODE HERE

## Submit a KFP run
After defining your KFP pipeline, you can test it by triggering a KFP run of your KFP pipeline. 

In [4]:
# Specify pipeline argument values

# A loose threshold to guarantee that the MAE resulted from HPO can pass the KFP condition check
loose_mae_threshold = 81

# A strict threshold to guarantee that the MAE resulted fro HPO can't pass the KFP condition check
strict_mae_threshold = 0.1

url = "https://raw.githubusercontent.com/yumoL/mlops_eng_course_datasets/master/intro/bike-demanding/train_full.csv"

arguments = {
    "url": url,
    "random_seed": 42,
    "hpo_trials": 81,
    "mae_threshold": loose_mae_threshold,
    "mlflow_tracking_uri": "http://mlflow.mlflow.svc.cluster.local:5000",
    "mlflow_s3_endpoint_url": "http://mlflow-minio-service.mlflow.svc.cluster.local:9000",
    "mlflow_experiment_name": "bike-notebook",
    "model_name": "bike-demand",
}

In [5]:
run_name = "bike-run"
kfp_experiment_name = "bike-experiment"

kfp_client.create_run_from_pipeline_func(
    pipeline_func=pipeline,
    run_name=run_name,
    experiment_name=kfp_experiment_name,
    arguments=arguments,
    enable_caching=False
)

RunPipelineResult(run_id=b143ace0-d653-44b9-916c-727efbd2bf27)

You can now go to [http://ml-pipeline-ui.local](http://ml-pipeline-ui.local) to check the running state of the KFP run. You can also go to [http://mlflow-server.local](http://mlflow-server.local) to check the created MLflow experiment and the registered model.

When the KFP run is completed, you should see the following:

The KFP run is completed successfully
<details>
    <summary>Example</summary>
    <img src="./images/kfp-run-complete.png" width=1000/>
</details>

<br />
Clicking on the "condition-1" tab you should also see the "train" and "deploy_model" components are executed.
<details>
    <summary>Example</summary>
    <img src="./images/kfp-run-complete2.png" width=1000/>
</details>

<br />
An MLflow Run is logged, including the logged hyperparameters and evaluation metrics.

<details>
    <summary>Example</summary>
    <img src="./images/mlflow-log.png" width=1000/>
</details>

<br />
A model corresponding to the MLflow Run is uploaded.

<details>
    <summary>Example</summary>
    <img src="./images/mlflow-model.png" width=1000/>
</details>

#### Capture screenshots
Please capture the following screenshots and put them into your PDF file:

1) The Kubeflow Pipelines UI of a succeeded KFP run of this KFP pipeline.

2) The MLflow UI of the created MLflow experiment, including the logged hyperparameters and evaluation metrics.

3) The MLflow UI of the registered model.

These screenshots should be similar to those given in the examples.  

#### A hint for debugging

Your KFP run may fail to complete because of some components' bugs. You can further investigate the problems by looking into the component logs:
(The screenshot was captured from an older version of KFP, the UI looks different in the current version. But the idea is the same.)

![](./images/kfp-debug.jpg)

## Testing the inference service
After the KFP run in completed, let's ensure that it's ready.

In [6]:
# The inference service should be immediately read when the KFP Run is completed
!kubectl -n kserve-inference get isvc bike-demand

NAME          URL                                               READY   PREV   LATEST   PREVROLLEDOUTREVISION   LATESTREADYREVISION           AGE
bike-demand   http://bike-demand.kserve-inference.example.com   True           100                              bike-demand-predictor-00003   3d14h


In [7]:
#  Send a request to the inference service
res = send_requests(isvc_name="bike-demand")
assert res.status_code == 200, "The inference service is not ready."
assert res.json()["outputs"][0]["shape"] == [2, 1], "The shape of the prediction is incorrect."
print(res.json())

{'model_name': 'bike-demand', 'id': '34882bd8-9107-43bf-a2ce-2a432f0e5b43', 'parameters': {}, 'outputs': [{'name': 'output-1', 'shape': [2, 1], 'datatype': 'FP64', 'data': [50.96589635634141, 34.33429439127524]}]}


Example output:
```text
{'model_name': 'bike-demand',
 'id': 'e85caa5a-3ebb-4b8d-a554-06d5d76ce462',
 'parameters': {},
 'outputs': [{'name': 'output-1',
   'shape': [2, 1],
   'datatype': 'FP64',
   'data': [35.894812901164, 31.72387585260099]}]}
```

In [8]:
# Delete the inference service
!kubectl -n kserve-inference delete isvc bike-demand

inferenceservice.serving.kserve.io "bike-demand" deleted


## Another run of the pipeline
Now let's use the "strict_mae_threshold" as the threshold metric and start another KFP run of your KFP pipeline. This time the KFP run will skip the "train" and "deploy_model" tasks because the best MAE found by the HPO component is larger than the strict threshold.
<details>
    <summary>Example</summary>
    <img src="./images/kfp-run-skip-train-and-deploy.png" width=1100/>
</details>

<br />
Clicking the "condition-1" tab you should see the "train" and "deploy_model" components are skipped.
<details>
    <summary>Example</summary>
    <img src="./images/kfp-run-skip-train-and-deploy2.png" width=1100/>
</details>

In [9]:
arguments["mae_threshold"] = strict_mae_threshold

run_name = "bike-run-incomplete"

kfp_client.create_run_from_pipeline_func(
    pipeline_func=pipeline,
    run_name=run_name,
    experiment_name=kfp_experiment_name,
    arguments=arguments,
    enable_caching=False
)

RunPipelineResult(run_id=0f86e7a5-0b8b-4e72-b61b-7b49d44264d4)

#### Capture screenshots
Like the examples, please **capture screenshots** of the KFP run where the deployment task is skipped and put this screenshot into your PDF file.

Finally, let's compile the KFP pipeline and save it to a YAML file `pipeline.yaml`. The file should be located in the same directory as this notebook.

In [10]:
compiler = kfp.compiler.Compiler()
compiler.compile(pipeline, "pipeline.yaml")

### Wrap-up
Please include the following files in your submission:
- The assignment notebooks (`week5_assignments_part1.ipynb` and `week5_assignments_part2.ipynb`)
- The YAML file of your KFP pipeline `pipeline.yaml`.
- The PDF containing the screenshots for Assignment 2. 